In [ ]:
#库

In [19]:
import numpy as np
import scipy
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt

In [20]:
import pandas as pd
data = pd.read_csv("数据集.csv")
data

,Date,Country,Product,Price,Revenue ($)
0,0,Indonesia,Product A1,44.63,2945721
1,0,Indonesia,Product A1,44.80,2957125
2,1,China,Product A1,38.33,1609860
3,1,China,Product A1,38.56,3855961
4,3,China,Product A1,38.22,2108330
...,...,...,...,...,...
303,280,Taiwan,Product A1,43.19,3730839
304,281,Taiwan,Product A1,43.19,3318720
305,282,China,Product B3,51.84,3659984
306,282,Taiwan,Product A1,43.19,3800677


In [ ]:
#处理x和y

In [88]:
x=data.drop('Product', axis=1)
y=data['Product']

In [ ]:
#解决过采样和欠采样问题

In [91]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_oversampled, y_oversampled = ros.fit_resample(x, y)
y_oversampled

0       Product A1
1       Product A1
2       Product A1
3       Product A1
4       Product A1
           ...    
1489    Product D2
1490    Product D2
1491    Product D2
1492    Product D2
1493    Product D2
Name: Product, Length: 1494, dtype: object

In [103]:
X_oversampled

,Date,Country,Price,Revenue ($)
0,0,Indonesia,44.63,2945721
1,0,Indonesia,44.80,2957125
2,1,China,38.33,1609860
3,1,China,38.56,3855961
4,3,China,38.22,2108330
...,...,...,...,...
1489,13,Hongkong,40.59,3074693
1490,13,Hongkong,40.59,3074693
1491,14,India,40.67,1016750
1492,8,China,44.00,3018400


In [104]:
x=X_oversampled.drop('Revenue ($)', axis=1)   #dataframe.drop('isrun', axis=1)
x['product']=y_oversampled
x.head()

,Date,Country,Price,product
0,0,Indonesia,44.63,Product A1
1,0,Indonesia,44.80,Product A1
2,1,China,38.33,Product A1
3,1,China,38.56,Product A1
4,3,China,38.22,Product A1


In [ ]:
#哑变量处理

In [105]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(x['Country'])

In [106]:
x['Country'] = label

In [107]:
label = le.fit_transform(x['product'])
x['product'] = label
x

,Date,Country,Price,product
0,0,4,44.63,0
1,0,4,44.80,0
2,1,1,38.33,0
3,1,1,38.56,0
4,3,1,38.22,0
...,...,...,...,...
1489,13,2,40.59,8
1490,13,2,40.59,8
1491,14,3,40.67,8
1492,8,1,44.00,8


In [ ]:
#数据标准化

In [108]:
from sklearn.preprocessing import MinMaxScaler
X_new = MinMaxScaler().fit_transform(x)
x = X_new

In [110]:
y=X_oversampled['Revenue ($)']
y.head()

0    2945721
1    2957125
2    1609860
3    3855961
4    2108330
Name: Revenue ($), dtype: int64

In [111]:
price_frame=pd.DataFrame(y)
price_frame

,Revenue ($)
0,2945721
1,2957125
2,1609860
3,3855961
4,2108330
...,...
1489,3074693
1490,3074693
1491,1016750
1492,3018400


In [112]:
y_new = MinMaxScaler().fit_transform(price_frame)
y_new

array([[0.47854849],
       [0.48052686],
       [0.2468032 ],
       ...,
       [0.14391041],
       [0.49115685],
       [0.14391041]])

In [113]:
y_new=y_new.tolist()
y_new

[[0.4785484920242524],
 [0.4805268590560947],
 [0.24680319550339588],
 [0.636457015968843],
 [0.33327781970213466],
 [0.4069515209867548],
 [0.4508992340853696],
 [0.20636359693634146],
 [0.45759972937104787],
 [0.5938333029743162],
 [0.4911568520301508],
 [0.39999878563931746],
 [0.4736443831481433],
 [0.5719482682349267],
 [0.5009225671584827],
 [0.39005039596832247],
 [0.17315048531057273],
 [0.14391041487765316],
 [0.577676060613946],
 [0.4233391449165994],
 [0.45809588244988586],
 [0.6236196622342501],
 [0.4007938449261408],
 [0.3407974880081882],
 [0.3550797574748237],
 [0.2454188243253793],
 [0.4561074535723889],
 [0.4400803212851405],
 [0.39669693894367963],
 [0.6068401467641624],
 [0.4950306626072324],
 [0.29041366329247875],
 [0.4947799838663509],
 [0.3137855959475049],
 [0.6371509363588261],
 [0.3410844240894463],
 [0.34139235126250145],
 [0.4216013947799838],
 [0.3985257661314805],
 [0.6480255362703513],
 [0.16373311821801242],
 [0.4784412813240001],
 [0.5108503126978757],


In [114]:
list2 = np.array(y_new).flatten().tolist()
y = list2

In [ ]:
#分测试集和训练集

In [115]:
seed=5
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=seed)

In [ ]:
#随机森林

In [116]:
model = RandomForestRegressor(n_estimators=10, random_state=123)
model.fit(xtrain,ytrain)

RandomForestRegressor(n_estimators=10, random_state=123)

In [117]:
print ('各feature的重要性：%s' % model.feature_importances_)

各feature的重要性：[0.21709516 0.078593   0.60843869 0.09587315]


In [118]:
result = model.score(xtest,ytest)    #导入测试集，rfc的接口score计算的是模型准确率accuracy
result

0.8026843197722063

In [ ]:
#线性回归

In [119]:
from sklearn.linear_model import LinearRegression
regr = LinearRegression()
regr.fit(xtrain,ytrain)
result = regr.score(xtest,ytest)    #导入测试集，rfc的接口score计算的是模型准确率accuracy
result

0.27101186275685585

In [ ]:
#adaboost

In [120]:
from sklearn.ensemble import AdaBoostRegressor
model = AdaBoostRegressor(random_state=123)
model.fit(xtrain,ytrain)
result = model.score(xtest,ytest)    #导入测试集，rfc的接口score计算的是模型准确率accuracy
result

0.477976169035389

In [ ]:
#决策树

In [121]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(max_depth=2, random_state=0)
model.fit(xtrain,ytrain)
result = model.score(xtest,ytest)    #导入测试集，rfc的接口score计算的是模型准确率accuracy
result

0.3772742467895721

In [ ]:
#k近邻算法

In [122]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=2)
model.fit(xtrain,ytrain)
result = model.score(xtest,ytest)    #导入测试集，rfc的接口score计算的是模型准确率accuracy
result

0.8317750438636176

In [ ]:
#简易mlp神经网络

In [123]:
from sklearn.neural_network import MLPRegressor
model = MLPRegressor(random_state=123)  # 设置random_state随机状态参数，使得每次训练的模型都是一样的
model.fit(xtrain,ytrain)
result = model.score(xtest,ytest)    #导入测试集，rfc的接口score计算的是模型准确率accuracy
result

0.3070570047950235